In [1]:
import pandas as pd
df = pd.read_csv('c:/data/dacon/card.csv')

In [2]:
sj = df[df['selng_cascnt']=='석재'].index
ct = df[df['selng_cascnt']=='커텐'].index
ys = df[df['selng_cascnt']=='영상물'].index
bj = df[df['selng_cascnt']=='복지매장'].index
ch = df[df['selng_cascnt']=='축협직영매장'].index

df_mod1 = df.drop(sj)
df_mod2 = df_mod1.drop(ct)
df_mod3 = df_mod2.drop(ys)
df_mod4 = df_mod3.drop(bj)
df_mod5 = df_mod4.drop(ch)

In [3]:
df_mod5['selng_cascnt_int'] = pd.to_numeric(df_mod5['selng_cascnt'])
df_mod5['receipt_dttm'] = df_mod5['receipt_dttm'].astype(str)
df_mod5['date']=df_mod5['receipt_dttm'].str[0:4]+"-"+df_mod5["receipt_dttm"].str[4:6]+"-"+df_mod5["receipt_dttm"].str[6:8]
df_mod5['date']=df_mod5['date'].astype('datetime64[ns]')

In [4]:
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt

In [18]:
def forcast_value(df, train, test, column):
    model=auto_arima(df[column],suppress_warnings=True)
    p,d,q = model.order
    print(f'({p},{d},{q})')
    global asw
    if model.order == (0,0,0):
        # 백색잡음 그래프일 경우 비시계열 자료이므로 분석에서 제외.
        print('백색잡음')

    elif p*q != 0:        
        # p, q 값이 둘다 0이 아닐 경우. 즉, AR과 MA 둘다 사용 가능한 경우
        # 추천된 order 값을 그대로 ARIMA 모형에 대입하였지만 정상성이 확보되지 않은 경우(차분으로 해결되지 않음) 에러 발생
        # 원래는 해당 그래프를 로그 변환 등의 정상화 과정을 거친 뒤 ARIMA 모형을 활용하는 것이 FM
        # 그러나 정상성이 전제되지 않아도 ARIMA 모형을 활용한 전례는 논문에도 나와 있음.
        # 그렇기 때문에 정상화 과정(배운적도 없으며 해당 이론 혹은 코드 찾는게 쉽지 않다) 모두 적용하는 것보다 order 값을 조정하는게 효율적이라는 판단.
        
        # 다시 돌아와서
        # p, q 값이 둘다 0이 아닐 경우. 즉, AR과 MA 둘다 사용 가능한 경우
        # 정상성 에러 이슈를 방지하기 위해 사전에 order 값을 조정한다
        # 심지어 AR 모형은 전반적인 추세, 회귀식을 도출하는 모형으로 정상성이 전제되지 않아도 되기 때문에
        # AR, MA 모형을 모두 활용 가능한 경우에는 AR 모형으로만 분석을 전개.
        q = 0
        model=ARIMA(train[column], order=(p,d,q)).fit()
        fc,se,conf=model.forecast(steps=len(test))
        fc_list = list(fc)
        fc_value = 0
        for i in range(-10,0,1):
            fc_value += (fc_list[i]-fc_list[i-1])/abs(fc_list[i-1])*100
        print(fc_value)
        
    else:        
        model=ARIMA(train[column], order=(p,d,q)).fit()
        fc,se,conf=model.forecast(steps=len(test))
        fc_list = list(fc)
        fc_value = 0
        for i in range(-10,0,1):
            fc_value += (fc_list[i]-fc_list[i-1])/abs(fc_list[i-1])*100
        print(fc_value)

In [19]:
from tqdm import tqdm_notebook
for i in tqdm_notebook(sorted(list(set(df_mod5['mrhst_induty_cl_code'])))):
    print(i)
    df = pd.DataFrame(df_mod5[df_mod5['mrhst_induty_cl_code'] == i].groupby('date').selng_cascnt_int.sum())
    if len(df) < 30:
        print('데이터가 부족합니다')
        
    else:
        n=int(df.shape[0]*0.8)
        train=df[:n]
        test=df[n:]
        forcast_value(df, train, test, 'selng_cascnt_int')
    print('-----------------------------------')

1001
(2,0,2)


C:\Users\analysis\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


-8.104719559607924e-09
-----------------------------------
1002
(0,0,1)


C:\Users\analysis\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


0.0
-----------------------------------
1003
(2,0,4)


...

1.3902363802216007
-----------------------------------
9911
(1,0,3)


C:\Users\analysis\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


1.8584355318044754e-11
-----------------------------------
9921
데이터가 부족합니다
-----------------------------------
9931
(1,1,3)


C:\Users\analysis\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\analysis\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


-75.87738101060312
-----------------------------------
9992
데이터가 부족합니다
-----------------------------------
9993
데이터가 부족합니다
-----------------------------------
9996
데이터가 부족합니다
-----------------------------------
9998
(0,0,0)
백색잡음
-----------------------------------

